# Working With DataRobot Deployments
### The examples below show examples of how you might manage model deployments

### Good eample reference on github:  
### https://github.com/datarobot-community/examples-for-data-scientists/blob/master/Making%20Predictions/Python/Batch%20Prediction%20API.ipynb

### **For batch and realtime scoring examples, please see:
- ./Deployment predictions - Batch.ipynb
- ./Deployment predictions - Real-time REST with helper functions.ipynb

In [17]:
import pandas as pd
import datarobot as dr
from datetime import datetime
from datarobot.enums import SERVICE_STAT_METRIC
from datarobot.helpers.partitioning_methods import construct_duration_string
import requests
import os
import time
from pprint import pprint as pp

In [18]:
# 
# Connect to DataRobot  
#
# Replace the API token and username with yours, along with the 
# depolyment id and/or project and model ids you are using.  
#

USERNAME = os.environ['DATAROBOT_USERNAME']
API_TOKEN = os.environ['DATAROBOT_API_TOKEN']
ENDPOINT = 'https://app.datarobot.com/api/v2'

dr.Client(token=API_TOKEN, endpoint=ENDPOINT)  

## Get Deployments

In [19]:
# List all of your deployments
deployments = dr.Deployment.list()
deployments

[Deployment(MMM Twitter messages polarity monitoring in Boston),
 Deployment(Housing Price Predictor (with Challengers)),
 Deployment(is_bad Predictions KLA),
 Deployment([MultiSeries] Bluebikes Availability (with challengers)),
 Deployment(MBTA Parking Utilization Forecasting [Deprecated]),
 Deployment(is_bad Predictions),
 Deployment(Store sales single series),
 Deployment(DRDemo_DP_ContinuousAI_NYC_CALLS-4Y_00),
 Deployment(DRDemo_DP_ContinuousAI_NYC311_00),
 Deployment(DRDemo_DP_ContinuousAI_HomeCredit_00),
 Deployment(DRDemo_DP_ContinuousAI_MEDICAL_AFD_00),
 Deployment(Weber Shandwick Sentiment Model - Multiclass),
 Deployment(CANCELED Predictions - ORIGINAL),
 Deployment(Lending Club Deployed Model),
 Deployment(is_bad Predictions MC),
 Deployment(lending club is_bad Predictions),
 Deployment([RAM-MODEL] Model Fit RAM Predictions),
 Deployment(Loan Defaults with Challengers [Segments]),
 Deployment(Hospital readmitted for diabetes MC demo e2e),
 Deployment(Pixability RoBERTA with

In [20]:
# Get deployments by search string
deployments_lc = dr.Deployment.list(search='is_bad Predictions')
deployments_lc

[Deployment(is_bad Predictions KLA),
 Deployment(is_bad Predictions),
 Deployment(is_bad Predictions MC),
 Deployment(lending club is_bad Predictions),
 Deployment(is_bad Predictions AMAT),
 Deployment(is_bad Predictions - notebook code example),
 Deployment(is_bad Predictions),
 Deployment(is_bad Predictions - replace notebook example),
 Deployment(is_bad Predictions),
 Deployment(is_bad Predictions),
 Deployment(is_bad Predictions),
 Deployment(is_bad Predictions),
 Deployment(is_bad Predictions test 1),
 Deployment(is_bad Predictions test 2),
 Deployment(is_bad Predictions - challenger test),
 Deployment(is_bad Predictions sdgsdgf),
 Deployment(is_bad Predictions),
 Deployment(is_bad Predictions),
 Deployment(is_bad Predictions M-D),
 Deployment(is_bad Predictions 1),
 Deployment(is_bad Predictions 2),
 Deployment(is_bad Predictions cambia),
 Deployment(is_bad Predictions 3),
 Deployment(is_bad Predictions PPS),
 Deployment(is_bad Predictions temp RS),
 Deployment(is_bad Predictions

In [21]:
# Get first first deployment
deployment_lc_first = deployments_lc[0]

# Get the most recent deployment
deployment_lc_last = deployments_lc[-1]
print(deployment_lc_last)

# Let's use the most recent model info from a deployment
deployment = deployment_lc_last
deployment.model

Deployment(is_bad Predictions)


{'id': '60ca49604acc6cd17d6a87cc',
 'type': 'Light Gradient Boosted Trees Classifier with Early Stopping',
 'target_name': 'is_bad',
 'project_id': '60ca463ee5e31e2109ecf5af',
 'target_type': 'Binary',
 'project_name': '_10K Lending Club Loans.csv',
 'unsupervised_mode': False,
 'unstructured_model_kind': False,
 'build_environment_type': 'DataRobot',
 'deployed_at': '2021-06-16T20:04:32.740000Z'}

## Deployment attributes and methods

In [22]:
for item in dir(deployment):
    print(item)
print()
deployment.__dict__

__class__
__delattr__
__dict__
__dir__
__doc__
__eq__
__format__
__ge__
__getattribute__
__gt__
__hash__
__init__
__init_subclass__
__le__
__lt__
__module__
__ne__
__new__
__reduce__
__reduce_ex__
__repr__
__setattr__
__sizeof__
__str__
__subclasshook__
__weakref__
_build_query_params
_capabilities
_client
_converter
_create_from_custom_model_entity
_default_prediction_server_converter
_fields
_filter_data
_health
_model_converter
_path
_prediction_usage
_safe_data
_server_data
accuracy_health
capabilities
create_from_custom_model_image
create_from_custom_model_version
create_from_learning_model
default_prediction_server
delete
description
download_prediction_results
download_scoring_code
from_data
from_location
from_server_data
get
get_accuracy
get_accuracy_over_time
get_association_id_settings
get_challenger_models_settings
get_drift_tracking_settings
get_feature_drift
get_features
get_prediction_intervals_settings
get_prediction_results
get_prediction_warning_settings
get_prediction

{'id': '60ca595078628dac87cd35cc',
 'label': 'is_bad Predictions',
 'description': None,
 'default_prediction_server': {'id': '5a61d7a0fbd723001a2f70d9',
  'url': 'https://cfds-ccm-prod.orm.datarobot.com',
  'datarobot-key': '544ec55f-61bf-f6ee-0caf-15c7f919a45d'},
 'model': {'id': '60ca49604acc6cd17d6a87cc',
  'type': 'Light Gradient Boosted Trees Classifier with Early Stopping',
  'target_name': 'is_bad',
  'project_id': '60ca463ee5e31e2109ecf5af',
  'target_type': 'Binary',
  'project_name': '_10K Lending Club Loans.csv',
  'unsupervised_mode': False,
  'unstructured_model_kind': False,
  'build_environment_type': 'DataRobot',
  'deployed_at': '2021-06-16T20:04:32.740000Z'},
 'capabilities': {'supports_model_replacement': True,
  'supports_target_drift_tracking': True,
  'supports_feature_drift_tracking': True,
  'supports_prediction_warning': False,
  'supports_prediction_intervals': False,
  'supports_humility_rules': True,
  'supports_humility_rules_default_calculations': True,
 

## Get service stats

In [27]:
# View deployment stats
service_stats = deployment.get_service_stats()
print(service_stats.metrics)

print(deployment.service_health)
print(deployment.model_health)
print(deployment.accuracy_health)

{'cacheHitRatio': 0.0,
 'executionTime': None,
 'medianLoad': 0.0,
 'numConsumers': 0,
 'peakLoad': 0.0,
 'responseTime': None,
 'serverErrorRate': 0.0,
 'slowRequests': 0,
 'totalPredictions': 0,
 'totalRequests': 0,
 'userErrorRate': 0.0}
{'status': 'unknown', 'start_date': datetime.datetime(2022, 4, 15, 0, 0, tzinfo=tzutc()), 'end_date': datetime.datetime(2022, 4, 16, 0, 0, tzinfo=tzutc())}
{'status': 'unavailable', 'message': '0 low-importance attribute distributions have drifted since the model was trained.'}
{'status': 'unavailable', 'message': 'Accuracy is unknown.'}


## Create a new Deployment

In [25]:
#
# Create new deployment from a project
#
dr.Project(deployment.model.get('project_id'))

# dr.Deployment.create...

Project(60ca463ee5e31e2109ecf5af)

## Replace a mode in a deployment

In [ ]:
# Replace a model with a new model from a new project
from datarobot.enums import MODEL_REPLACEMENT_REASON

deployment = dr.Deployment.list(search=DEPLOYMENT_NAME)[0]
print(deployment.service_health)
print(deployment.model_health)
print(deployment.accuracy_health)

retrain_project = dr.Project.start(sourcedata=TRAIN_FILE, 
                                      project_name=NEW_PROJECT_NAME, target=TARGET, 
                                      autopilot_on=True,
                                      worker_count=-1)

model = retrain_project.get_models()[0]  # order is the leaderboard, so 0 is the top

deployment.replace_model(model.id, MODEL_REPLACEMENT_REASON.ACCURACY)

In [48]:
# status (array[string]) – (New in version v2.25) Optional. Filters deployments by their status
# enum: [‘inactive’, ‘archived’, ‘active’]
    
# Set the status:
# https://app.datarobot.com/apidocs/entities/deployments.html#post--api-v2-deployments-fromLearningModel-

# Get the status
# https://app.datarobot.com/apidocs/entities/deployments.html#get--api-v2-deployments-(deploymentId)-

In [49]:
def _raise_dataroboterror_for_status(response):
    """Raise DataRobotPredictionError if the request fails along with the response returned"""
    try:
        response.raise_for_status()
    except requests.exceptions.HTTPError:
        err_msg = '{code} Error: {msg}'.format(
            code=response.status_code, msg=response.text)
        raise DataRobotPredictionError(err_msg)

class DataRobotPredictionError(Exception):
    """Raised if there are issues getting predictions from DataRobot"""

    
headers = {
    # As default, we expect CSV as input data.
    # Should you wish to supply JSON instead,
    # comment out the line below and use the line after that instead:
    'Content-Type': CONTENT_TYPE,
    # 'Content-Type': 'application/json; charset=UTF-8',

    'Authorization': 'Bearer {}'.format(API_KEY),
    'DataRobot-Key': DATAROBOT_KEY,
}

url = API_URL.format(deployment_id=deployment.id)

# Prediction Explanations:
# See the documentation for more information:
# https://app.datarobot.com/docs/predictions/api/dr-predapi.html#request-pred-explanations
# Should you wish to include Prediction Explanations or Prediction Warnings in the result,
# Change the parameters below accordingly, and remove the comment from the params field below:

params = {
    # If explanations are required, uncomment the line below
    # 'maxExplanations': 3,
    # 'thresholdHigh': 0.5,
    # 'thresholdLow': 0.15,
    # Uncomment this for Prediction Warnings, if enabled for your deployment.
    # 'predictionWarningEnabled': 'true',
}
# Make API request for predictions

print(url)

predictions_response = requests.get(
    url,
#     data=data,
    headers=headers,
    # Prediction Explanations:
    # Uncomment this to include explanations in your prediction
    # params=params,
)
_raise_dataroboterror_for_status(predictions_response)
# Return a Python dict following the schema in the documentation

predictions_response.json()

https://mlops.dynamic.orm.datarobot.com


DataRobotPredictionError: 404 Error: <html>
<head><title>404 Not Found</title></head>
<body>
<center><h1>404 Not Found</h1></center>
<hr><center>nginx</center>
</body>
</html>


## Get service stats

In [12]:
ss = d.get_service_stats()
ss.metrics

{'totalPredictions': 31929468,
 'userErrorRate': 0.00102897373409152,
 'cacheHitRatio': 0.881884646628757,
 'executionTime': 145.5,
 'totalRequests': 18465,
 'serverErrorRate': 0,
 'slowRequests': 1719,
 'medianLoad': 2,
 'numConsumers': 1,
 'responseTime': 495.666666666667,
 'peakLoad': 2}

In [67]:
# ss = d.get_service_stats()
ss = d.get_service_stats(
    start_time=datetime(2020, 8, 1, hour=0),
    end_time=datetime(2020, 11, 15, hour=23),
)
# ss[SERVICE_STAT_METRIC.ALL]
ss[SERVICE_STAT_METRIC.TOTAL_PREDICTIONS]
[a for a in dir(ss) if '__' not in a]

['_build_query_params',
 '_client',
 '_converter',
 '_fields',
 '_filter_data',
 '_path',
 '_period',
 '_safe_data',
 '_server_data',
 'from_data',
 'from_location',
 'from_server_data',
 'get',
 'metrics',
 'model_id',
 'period']

In [68]:
ss.period

{'start': datetime.datetime(2020, 8, 1, 0, 0, tzinfo=tzutc()),
 'end': datetime.datetime(2020, 11, 15, 23, 0, tzinfo=tzutc())}

In [69]:
ss.metrics

{'totalPredictions': 361449,
 'userErrorRate': 0.000711842150730848,
 'cacheHitRatio': 0.999585334669477,
 'executionTime': 72,
 'totalRequests': 144695,
 'serverErrorRate': 0.000442309685891012,
 'slowRequests': 759,
 'medianLoad': 1,
 'numConsumers': 4,
 'responseTime': 115,
 'peakLoad': 10}

In [57]:
total_predictions = d.get_service_stats_over_time(
    start_time=datetime(2020, 8, 1, hour=15),
    end_time=datetime(2020, 11, 8, hour=15),
    bucket_size=construct_duration_string(days=1)
)
total_predictions.metric

'totalPredictions'

In [56]:
ss_ot = d.get_service_stats_over_time(
    start_time=datetime(2020, 8, 1, hour=15),
    end_time=datetime(2020, 11, 8, hour=15),
    bucket_size=construct_duration_string(days=1),
    metric=SERVICE_STAT_METRIC.TOTAL_PREDICTIONS
)
ss_ot.metric

'totalPredictions'

In [55]:
[a for a in dir(ss_ot) if '__' not in a]

['_bucket',
 '_build_query_params',
 '_client',
 '_converter',
 '_fields',
 '_filter_data',
 '_path',
 '_period',
 '_safe_data',
 '_server_data',
 'bucket_values',
 'buckets',
 'from_data',
 'from_location',
 'from_server_data',
 'get',
 'metric',
 'model_id',
 'summary']

In [53]:
#  'bucket_values',
#  'buckets',
#  'from_data',
#  'from_location',
#  'from_server_data',
#  'get',
#  'metric',
#  'model_id',
#  'summary']
total_predictions.bucket_values
total_predictions.buckets
total_predictions.metric
# total_predictions.model_id
# total_predictions.summary

'totalPredictions'

## End to end example workflow

In [6]:
# ******************************
# An example end to end workflow
# ******************************

# Get a project
most_recent_project = dr.Project.list()[-1]
print(most_recent_project)

# Get models
models = most_recent_project.get_models()
best_model = models[0]
print(best_model)

# Get a list of the prediction servers and use the first one
prediction_servers = dr.PredictionServer.list()
pred_server_url = prediction_servers[0]

# Deploy model
new_deployment = dr.Deployment.create_from_learning_model(model_id=best_model.id, 
                                                          label='My New Deployment',
                                                          default_prediction_server_id=pred_server_url.id)
print('\nThe new deployment and its model')
print(new_deployment)
print(new_deployment.model.get('id'))
print(new_deployment.model.get('type'))

# Get another model
model_2 = models[1]

# replace the model in the deployment with the new model
new_deployment.replace_model(model_2.id, reason=dr.enums.MODEL_REPLACEMENT_REASON.ACCURACY)
print('\nThe new deployment and its replaced model')
print(new_deployment.model.get('id'))
print(new_deployment.model.get('type'))

# Delete the deployment
new_deployment.delete()

Project(early_2012_2013_train.csv.zip)
Model('Elastic-Net Classifier (L2 / Binomial Deviance)')

The new deployment and its model
Deployment(My New Deployment)
5ab534695153e09e03250fc0
Elastic-Net Classifier (L2 / Binomial Deviance)

The new deployment and its replaced model
5ab535785153e0fda98c84b5
ENET Blender
